# Base de datos diario -ICC

**Objetivo**
Con este notebook se reproduce el excel que ICC maneja como insumo para sus calculos, reporte y demas.

In [1]:
# check donde estamos trabajando
#pwd

In [2]:
# libreria para movernos entre diferentes rutas
#import os

#os.chdir('../../')

In [3]:
# Libreías generales
import numpy as np
import pandas as pd
import math
import datetime
from dateutil.relativedelta import relativedelta
import glob
import datetime

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
import matplotlib.font_manager
from matplotlib import style
style.use('ggplot') or plt.style.use('ggplot')
import seaborn as sns



# Preprocesado y modelado
# ==============================================================================
from sklearn.cluster import DBSCAN, KMeans
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors


import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

In [4]:
#comando para visualizar todas las columnas del df
pd.pandas.set_option('display.max_columns', None)

# features load
los datos aqui cargados sufireron una limpieza en el notebook 

In [5]:
# get data file names
path =r'/home/diana/Escritorio/CIAT/ICC/historicos_horarios_excel/'
filenames = glob.glob(path + "/*.xlsx")
filenames

['/home/diana/Escritorio/CIAT/ICC/historicos_horarios_excel/san_rafael.xlsx',
 '/home/diana/Escritorio/CIAT/ICC/historicos_horarios_excel/la_giralda.xlsx',
 '/home/diana/Escritorio/CIAT/ICC/historicos_horarios_excel/concepcion.xlsx',
 '/home/diana/Escritorio/CIAT/ICC/historicos_horarios_excel/santa_teresa.xlsx',
 '/home/diana/Escritorio/CIAT/ICC/historicos_horarios_excel/el_carmen.xlsx',
 '/home/diana/Escritorio/CIAT/ICC/historicos_horarios_excel/central_izalco.xlsx',
 '/home/diana/Escritorio/CIAT/ICC/historicos_horarios_excel/xoluta.xlsx',
 '/home/diana/Escritorio/CIAT/ICC/historicos_horarios_excel/bougambilia.xlsx',
 '/home/diana/Escritorio/CIAT/ICC/historicos_horarios_excel/copal.xlsx',
 '/home/diana/Escritorio/CIAT/ICC/historicos_horarios_excel/trinidad.xlsx',
 '/home/diana/Escritorio/CIAT/ICC/historicos_horarios_excel/trinidad_magdalena.xlsx',
 '/home/diana/Escritorio/CIAT/ICC/historicos_horarios_excel/irlanda.xlsx',
 '/home/diana/Escritorio/CIAT/ICC/historicos_horarios_excel/agua

In [6]:
%%time
dfs = pd.DataFrame()
li=[]

for csv in filenames:
    df = pd.read_excel(csv, index_col=None)
    li.append(df)

dfs = pd.concat(li, axis=0, ignore_index=True)
dfs.head()

CPU times: user 13min 21s, sys: 3.22 s, total: 13min 25s
Wall time: 13min 25s


,Unnamed: 0,estacion,fecha,temperatura,radiacion,humedad_relativa,precipitacion,velocidad_viento,mojadura,direccion_viento,presion_atm
0,0,PANSRF,2010-02-16 00:14:59.515,24.3,0.0,87.0,0.0,0.0,10.0,89.7,NaN
1,1,PANSRF,2010-02-16 00:29:59.510,24.3,0.0,89.0,0.0,0.0,10.0,83.5,NaN
2,2,PANSRF,2010-02-16 00:44:59.505,23.2,0.0,90.0,0.0,0.0,10.0,101.7,NaN
3,3,PANSRF,2010-02-16 01:00:00.000,23.0,0.0,91.0,0.0,0.0,10.0,124.5,NaN
4,4,PANSRF,2010-02-16 01:15:00.000,22.7,0.0,92.0,0.0,0.0,10.0,134.5,NaN


In [7]:
dfs.drop('Unnamed: 0',axis=1, inplace=True)

In [8]:
dfs

,estacion,fecha,temperatura,radiacion,humedad_relativa,precipitacion,velocidad_viento,mojadura,direccion_viento,presion_atm
0,PANSRF,2010-02-16 00:14:59.515,24.3,0.0,87.0,0.0,0.0,10.0,89.7,NaN
1,PANSRF,2010-02-16 00:29:59.510,24.3,0.0,89.0,0.0,0.0,10.0,83.5,NaN
2,PANSRF,2010-02-16 00:44:59.505,23.2,0.0,90.0,0.0,0.0,10.0,101.7,NaN
3,PANSRF,2010-02-16 01:00:00.000,23.0,0.0,91.0,0.0,0.0,10.0,124.5,NaN
4,PANSRF,2010-02-16 01:15:00.000,22.7,0.0,92.0,0.0,0.0,10.0,134.5,NaN
...,...,...,...,...,...,...,...,...,...,...
11501038,SDTCBR,2022-08-14 22:44:59.898,25.0,0.0,100.0,0.6,5.6,10.0,71.1,NaN
11501039,SDTCBR,2022-08-14 22:59:59.898,24.7,0.0,100.0,5.2,7.0,10.0,40.9,NaN
11501040,SDTCBR,2022-08-14 23:14:59.898,23.5,0.0,100.0,3.6,1.5,10.0,52.7,NaN
11501041,SDTCBR,2022-08-14 23:29:59.898,23.4,0.0,100.0,0.0,0.0,10.0,321.7,NaN


pd.read_parquet('icc_stations_all.parquet')


# Agregacion diaria

In [9]:
dfs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11501043 entries, 0 to 11501042
Data columns (total 10 columns):
 #   Column            Dtype         
---  ------            -----         
 0   estacion          object        
 1   fecha             datetime64[ns]
 2   temperatura       float64       
 3   radiacion         float64       
 4   humedad_relativa  float64       
 5   precipitacion     float64       
 6   velocidad_viento  float64       
 7   mojadura          float64       
 8   direccion_viento  float64       
 9   presion_atm       float64       
dtypes: datetime64[ns](1), float64(8), object(1)
memory usage: 877.5+ MB


In [10]:
def segmentacion(df, col_fecha):
    """
    función que segmenta la fecha en dia, mes, año y semana.
    
    Arguments:
        df: dataframe 
        col_fecha: columna que contiene la informacion de la fecha
        
    Returns:
        df: dataframe de entrada con 5 nuevas columnas que informan sobre laç
        fecha dd/mm/aa, semana, dia, mes y año
    """
    df['date'] = pd.to_datetime(df[col_fecha]).dt.date
    df['semana'] = pd.to_datetime(df[col_fecha]).dt.week
    df['dia'] = pd.to_datetime(df[col_fecha]).dt.day
    df['mes'] = pd.to_datetime(df[col_fecha]).dt.month
    df['año'] = pd.to_datetime(df[col_fecha]).dt.year
    return df

In [11]:
#aqui se llama la funcion y se pone los parametros
segmentacion(dfs, 'fecha')

,estacion,fecha,temperatura,radiacion,humedad_relativa,precipitacion,velocidad_viento,mojadura,direccion_viento,presion_atm,date,semana,dia,mes,año
0,PANSRF,2010-02-16 00:14:59.515,24.3,0.0,87.0,0.0,0.0,10.0,89.7,NaN,2010-02-16,7,16,2,2010
1,PANSRF,2010-02-16 00:29:59.510,24.3,0.0,89.0,0.0,0.0,10.0,83.5,NaN,2010-02-16,7,16,2,2010
2,PANSRF,2010-02-16 00:44:59.505,23.2,0.0,90.0,0.0,0.0,10.0,101.7,NaN,2010-02-16,7,16,2,2010
3,PANSRF,2010-02-16 01:00:00.000,23.0,0.0,91.0,0.0,0.0,10.0,124.5,NaN,2010-02-16,7,16,2,2010
4,PANSRF,2010-02-16 01:15:00.000,22.7,0.0,92.0,0.0,0.0,10.0,134.5,NaN,2010-02-16,7,16,2,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11501038,SDTCBR,2022-08-14 22:44:59.898,25.0,0.0,100.0,0.6,5.6,10.0,71.1,NaN,2022-08-14,32,14,8,2022
11501039,SDTCBR,2022-08-14 22:59:59.898,24.7,0.0,100.0,5.2,7.0,10.0,40.9,NaN,2022-08-14,32,14,8,2022
11501040,SDTCBR,2022-08-14 23:14:59.898,23.5,0.0,100.0,3.6,1.5,10.0,52.7,NaN,2022-08-14,32,14,8,2022
11501041,SDTCBR,2022-08-14 23:29:59.898,23.4,0.0,100.0,0.0,0.0,10.0,321.7,NaN,2022-08-14,32,14,8,2022


# Agregaciones diarias
con la siguiente linea se construyen las columnas agregadas que tenia carlos en el excel

In [12]:
df_diario = dfs.groupby(['estacion', 'date','semana','dia','mes','año']).agg(
    temperatura_min_diaria=('temperatura', 'min'),
    temperatura_max_diaria=('temperatura', 'max'),
    temperatura_promedio_diaria=('temperatura', 'mean'),
    radiacion_diaria_acumulada=('radiacion', 'sum'),
    radiacion_diaria_promedio = ('radiacion','mean'),
    humedad_relativa_min_diaria = ('humedad_relativa', 'min'),
    humedad_relativa_max_diaria = ('humedad_relativa', 'max'),
    humedad_relativa_media_diaria = ('humedad_relativa', 'mean'),
    lluvia_diaria = ('precipitacion', 'sum'),
    velocidad_viento_media_diario = ('velocidad_viento', 'mean'),
    velocidad_viento_max_diaria = ('velocidad_viento', 'max')
).reset_index()

In [13]:
df_diario['amplitud_termica'] = df_diario.temperatura_max_diaria - df_diario.temperatura_min_diaria

# constantes para algunos calculos de variables
las tablas que se consultan fueron sacadas del excel compartido por ICC. se esta adelantando la revision bibliografica para generalizar estas tablas a otras estaciones.

In [15]:
"""
constantes de radiacion extraterreste y numero de dias despejado
"""

ra_n_ctes = pd.read_excel('tabla_Ra_N.xlsx').iloc[:-1]
ra_n_ctes.pop("Unnamed: 22")

ctes_radiacion = pd.read_excel('constantes_radiacion.xlsx')
ctes_radiacion.head()

,dia,1_rgl,1_rdp,2_rgl,2_rdp,3_rgl,3_rdp,4_rgl,4_rdp,5_rgl,5_rdp,6_rgl,6_rdp,7_rgl,7_rdp,8_rgl,8_rdp,9_rgl,9_rdp,10_rgl,10_rdp,11_rgl,11_rdp,12_rgl,12_rdp
0,1,336.3,254.1,363.8,274.9,402.3,304.0,434.3,328.1,445.5,336.6,443.9,335.4,442.0,334.0,442.2,334.1,434.1,328.0,408.6,308.7,369.5,279.2,339.5,256.5
1,2,336.7,254.4,365.1,275.8,403.6,305.0,435.0,328.7,445.5,336.7,443.8,335.3,442.0,334.0,442.2,334.1,433.6,327.6,407.4,307.9,368.2,278.2,338.9,256.1
2,3,337.2,254.8,366.3,276.8,404.9,305.9,435.6,329.2,445.6,336.7,443.7,335.2,442.0,334.0,442.1,334.1,433.0,327.2,406.3,307.0,367.0,277.3,338.3,255.6
3,4,337.8,255.2,367.6,277.7,406.1,306.9,436.3,329.6,445.7,336.7,443.6,335.2,442.0,334.0,442.0,334.0,432.4,326.7,405.1,306.1,365.7,276.4,337.8,255.2
4,5,338.3,255.6,368.9,278.7,407.4,307.8,436.9,330.1,445.7,336.8,443.5,335.1,442.0,334.0,441.9,333.9,431.8,326.3,403.9,305.2,364.5,275.4,337.2,254.8


In [18]:
ctes_location = pd.read_excel('estra_location.xlsx')
ctes_location['latitud_rad'] = ctes_location['latitud'].apply(lambda row: math.radians(row))

In [19]:
def extraer_info(df, filtros_str, name_feature, name_col):
    """
    Como la tabla tiene la informacion de varias constantes
    de interes como por ejemplo: radiacion general y radiacion en dia p,
    se hace un filtro para separalas. como hacemos el filtro nos queda en las columnas los meses y en las filas con el dia.
    es necesario quitar  el sufijo lpara solo quedarnos con el numero de meses.

    la matriz anterior se re diseña para que el mes ya no quede como columna sino como filas
    y los valores de la radiacion no quede en la matriz sino en una columna
    
    
    Arguments:
        df: dataframe 
        col_fecha: columna que contiene la informacion de la fecha
        
    Returns:
        df: dataframe de entrada con 5 nuevas columnas que informan sobre laç
        fe

    """
    matriz=df.filter(regex=filtros_str)
    matriz.columns = matriz.columns.str.replace("_" + name_feature, "")
    df_feature = matriz.melt(id_vars=["dia"], 
        var_name="mes", 
        value_name=name_col)
    df_feature.mes = df_feature.mes.astype('int')
    return df_feature

In [20]:
df_n = extraer_info(ra_n_ctes, 'dia|N', 'N', 'N Daylight hours')
df_ra = extraer_info(ra_n_ctes, 'dia|Ra', 'Ra', 'Ra')

In [21]:
df_rg = extraer_info(ctes_radiacion, 'rgl|dia', 'rgl', 'radiacion_global')
df_rdp = extraer_info(ctes_radiacion, 'rdp|dia', 'rdp', 'radiacion_dia_despejado')

## Join all dfs

In [22]:
"""
se unen los 3 dataframes donde las columnas comunes o las llaves son dia y mes y se 
reescribe el df_diario

"""
df_diario  = df_diario.merge(df_rg, how='inner', on=['dia', 'mes']).merge(df_rdp, how='inner', on=['dia', 'mes'])
df_diario = df_diario.merge(df_n, how='inner', on=['dia', 'mes']).merge(df_ra, how='inner', on=['dia', 'mes'])
##
df_diario = df_diario.merge(ctes_location, how= 'inner', on=['estacion'])

## Calculo de variables secundarias (las variables que requieren calculos)

In [25]:
def secundary_variables(df):
    df['radiacion_media_estimada Heargreaves'] = 0.16 * (
        np.sqrt(df.amplitud_termica)
    ) * df.radiacion_global
    
    df['Rg'] = 0.00089681 * df.radiacion_diaria_acumulada
    
    df['n']=(
        -0.32 +1.61*(df.Rg / df.Ra)
    )*df['N Daylight hours']
    
    ## balance hidrico
    
    df['dia_juliano'] = df['date'].apply(
        lambda row: (row -datetime.date(row.year,1,1)).days +1
    )
    
    df['dr'] = 1 + (0.033 * np.cos(((2*3.1416)/365)*df['dia_juliano']))
    
    df['declinacion'] = 0.409  * np.sin(
        (((2*3.1416)/365)*df['dia_juliano'])-1.39
    )
    
    df['ang_horario_puesta_sol'] = np.arccos(
        -(np.tan(df.latitud_rad) * np.tan(df.declinacion))
    )
    
    df['heliofania'] = (24/3.1416) * df['ang_horario_puesta_sol']
    
    df['ra_MJm-2day-1'] = ((24*60)/3.1416) * (0.082*df['dr']) * (
        (
            (df.ang_horario_puesta_sol * np.sin(df.latitud_rad))*(np.sin(df.declinacion))
        )+(
            np.cos(df.latitud_rad)*np.cos(df.declinacion)*np.sin(df.ang_horario_puesta_sol)
        )
    )
    
    df['radiacion_solar_piranometro'] = 0.0864 * df.radiacion_diaria_promedio
    
    df['constante_psicometrica'] =(
        (
            pow(((293-(0.0065 * df.altitud))/293), 5.26)
        )*101.3
    )*0.000665
    
    df['pendiente_curva_presion_satu_vapor'] = (
        4089 * (
            (0.6108 * (
                np.exp((17.27*df.temperatura_promedio_diaria)/(df.temperatura_promedio_diaria+237.3))
            )) / pow(
                (df_diario.temperatura_promedio_diaria+273.3),
                2)
        )
    )
    
    df['presion_saturacion_vapor'] = (
        (
            0.6108 * np.exp(
                (17.27*df.temperatura_min_diaria)/(df.temperatura_min_diaria +237.3)
            )
        )+(
            0.6108*np.exp(
                (17.27*df.temperatura_max_diaria)/(df.temperatura_max_diaria+237.3)
            )
        )
    )/2
    
    
    df['presion_real_vapor'] = (
        (
            0.6108*np.exp(
                (17.27*df.temperatura_max_diaria)/(df.temperatura_max_diaria +237.3)
            )
        ) * (df.humedad_relativa_min_diaria /100) + (
            0.6108*np.exp(
                (17.27*df.temperatura_min_diaria) / (df.temperatura_min_diaria+273.3)
            )
        ) * (df.humedad_relativa_max_diaria/100)
    )/2
    
    df['deficit_presion_vapor'] = df['presion_saturacion_vapor'] - df['presion_real_vapor']
    
    df['rns'] = df.radiacion_solar_piranometro * 0.77
    
    df['rnl']= (
        (
            (
                (
                    0.000000004903*(pow((df.temperatura_min_diaria+273.16),4))
                ) + (
                    (0.000000004903 * (pow(
                        (df.temperatura_max_diaria+273.16)
                        ,4)))
                )
            ) / 2
        ) * (
            0.34 -(0.14 *np.sqrt(df.presion_real_vapor))
        ) * (
            1.35 * (
                df.radiacion_solar_piranometro /((0.75+2*(df.altitud/100000))* df['ra_MJm-2day-1'])
            ) -0.35
        )
    )
    
    df['rn'] = df['rns'] - df['rnl']
    
    df['velocidad_viento_altura_standar'] = (
        (4.87)/((np.log((67.8*10)-5.42)))
    )*(
        (df.velocidad_viento_media_diario * 1000)/3600
    )
    
    df['FAO-Penman-Montieth'] = (
        (0.408 * df.pendiente_curva_presion_satu_vapor * df.rn) + (
            df.constante_psicometrica * (
                900/(df.temperatura_promedio_diaria + 273)
            )* df.velocidad_viento_altura_standar * df.deficit_presion_vapor
        )
    ) / (
        df.pendiente_curva_presion_satu_vapor + (
            df.constante_psicometrica + (
                1+(0.34 + df.velocidad_viento_altura_standar)
            )
        )
    )
    
    df['deficit']= df.lluvia_diaria - df['FAO-Penman-Montieth']
    return df

In [26]:
secundary_variables(df_diario)

,estacion,date,semana,dia,mes,año,temperatura_min_diaria,temperatura_max_diaria,temperatura_promedio_diaria,radiacion_diaria_acumulada,radiacion_diaria_promedio,humedad_relativa_min_diaria,humedad_relativa_max_diaria,humedad_relativa_media_diaria,lluvia_diaria,velocidad_viento_media_diario,velocidad_viento_max_diaria,amplitud_termica,radiacion_global,radiacion_dia_despejado,N Daylight hours,Ra,estacion_resume,altitud,estrato,latitud,longitud,latitud_rad,radiacion_media_estimada Heargreaves,Rg,n,dia_juliano,dr,declinacion,ang_horario_puesta_sol,heliofania,ra_MJm-2day-1,radiacion_solar_piranometro,constante_psicometrica,pendiente_curva_presion_satu_vapor,presion_saturacion_vapor,presion_real_vapor,deficit_presion_vapor,rns,rnl,rn,velocidad_viento_altura_standar,FAO-Penman-Montieth,deficit
0,CENCEN,2008-06-11,24,11,6,2008,19.4,31.9,25.094792,24413.60000,254.308333,52.0,93.0,76.666667,2.6,4.514583,21.1,12.5,442.9,334.7,12.828,38.185,CEN,300,Alto,14.330962,-91.055468,0.250122,250.542075,21.894361,7.737019,163,0.968842,0.404105,1.680265,12.836249,38.200705,21.972240,0.065039,0.146299,3.490402,2.121480,1.368922,16.918625,3.611187,13.307438,0.937969,0.420380,2.179620
1,CENCEN,2009-06-11,24,11,6,2009,21.2,32.7,25.609375,26112.40000,272.004167,58.0,96.0,81.541667,9.4,7.118750,29.6,11.5,442.9,334.7,12.828,38.185,CEN,300,Alto,14.330962,-91.055468,0.250122,240.311516,23.417861,8.561034,162,0.969033,0.402959,1.679917,12.833590,38.208086,23.501160,0.065039,0.150321,3.731921,2.450758,1.281163,18.095893,3.604390,14.491503,1.479021,0.415315,8.984685
2,CENCEN,2010-06-11,23,11,6,2010,22.1,32.5,26.316667,22499.40000,234.368750,53.0,95.0,78.062500,1.2,3.218750,20.0,10.4,442.9,334.7,12.828,38.185,CEN,300,Alto,14.330962,-91.055468,0.250122,228.529533,20.177687,6.808524,162,0.969033,0.402959,1.679917,12.833590,38.208086,20.249460,0.065039,0.156002,3.775434,2.352183,1.423251,15.592084,2.990995,12.601089,0.668741,0.443173,0.756827
3,CENCEN,2011-06-11,23,11,6,2011,22.1,32.7,26.965625,19675.10000,204.948958,57.0,96.0,78.562500,4.8,2.257292,10.1,10.6,442.9,334.7,12.828,38.185,CEN,300,Alto,14.330962,-91.055468,0.250122,230.716469,17.644826,5.438578,162,0.969033,0.402959,1.679917,12.833590,38.208086,17.707590,0.065039,0.161373,3.803104,2.476888,1.326216,13.634844,2.290996,11.343848,0.468984,0.426577,4.373423
4,CENCEN,2012-06-11,24,11,6,2012,22.2,32.5,26.411458,17912.70000,186.590625,53.0,91.0,76.812500,0.0,2.130208,21.1,10.3,442.9,334.7,12.828,38.185,CEN,300,Alto,14.330962,-91.055468,0.250122,227.428180,16.064288,4.583714,163,0.968842,0.404105,1.680265,12.836249,38.200705,16.121430,0.065039,0.156777,3.783556,2.313200,1.470357,12.413501,2.054312,10.359189,0.442581,0.394058,-0.394058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108865,TULTLA,2021-06-10,23,10,6,2021,22.6,30.7,25.122917,15399.80000,160.414583,74.0,100.0,95.250000,10.2,3.294792,10.3,8.1,443.0,334.7,12.825,38.193,TUL,253,Medio,14.506967,-91.586101,0.253194,201.728016,13.810695,3.362464,161,0.969234,0.401694,1.680932,12.841347,38.261462,13.859820,0.065399,0.146516,3.579305,2.776215,0.803090,10.672061,1.259913,9.412148,0.684540,0.300110,9.899890
108866,TULTLA,2022-06-10,23,10,6,2022,22.2,31.6,25.835417,18359.91000,191.249063,65.0,100.0,92.625000,2.4,4.806250,15.0,9.4,443.0,334.7,12.825,38.193,TUL,253,Medio,14.506967,-91.586101,0.253194,217.313965,16.465351,4.797649,161,0.969234,0.401694,1.680932,12.841347,38.261462,16.523919,0.065399,0.152117,3.662342,2.628454,1.033888,12.723418,1.899232,10.824186,0.998567,0.342374,2.057626
108867,TULTLA,2012-02-29,9,29,2,2012,19.9,34.1,25.833333,19328.00000,201.333333,39.0,97.0,72.531250,0.0,6.373958,25.8,14.2,401.0,303.0,11.714,34.531,TUL,253,Medio,14.506967,-91.586101,0.253194,241.773405,17.333544,5.718453,60,1.016908,-0.142987,1.533536,11.715325,34.507522,17.395200,0.065399,0.152101,3.836397,1.999572,1.836826,

In [27]:
df_diario.to_parquet('diario_complete.parquet')

# el raound es para el redondeo
df_diario['radiacion_media_estimada Heargreaves'] = round(
    0.16 * (np.sqrt(df_diario.amplitud_termica)) * df_diario.radiacion_global,
    1
)


df_diario['Rg']=0.00089681*df_diario.radiacion_diaria_acumulada

df_diario['n']=round(
    (-0.32 +1.61*(df_diario['Rg']/df_diario['Ra']))*df_diario['N Daylight hours'],
    1
)

In [ ]:


secundary_variables(df_diario)

In [ ]:
df_diario.head()

In [ ]:
def

In [ ]:
df_diario.to_parquet('df_diario.parquet')

# Balance hidrico

In [ ]:
df_diario = pd.read_parquet('df_diario.parquet')

In [ ]:
df_diario.head()

df_diario['dia_juliano'] = df_diario['date'].apply(
    lambda row: (row -datetime.date(row.year,1,1)).days +1
)

df_diario['dr'] = 1+(0.033*np.cos(((2*3.1416)/365)*df_diario['dia_juliano']))

df_diario['declinacion'] = 0.409*np.sin((((2*3.1416)/365)*df_diario['dia_juliano'])-1.39)

# lectura sobre altitud y latitud
A partir de aqui solo se quedan las 27 estaciones que analizan en ICC. antes habian 32

ctes_location = pd.read_excel('estra_location.xlsx')
ctes_location

ctes_location['latitud_rad'] = ctes_location['latitud'].apply(lambda row: math.radians(row))

df_diario = df_diario.merge(ctes_location, how= 'inner', on=['estacion'])

df_diario['ang_horario_puesta_sol'] = np.arccos(
    -(
        np.tan(df_diario.latitud_rad) * np.tan(df_diario.declinacion)
    )
)

df_diario['heliofania'] =(24/3.1416)*df_diario['ang_horario_puesta_sol']

df_diario['ra_MJm-2day-1'] = (
    (24*60)/3.1416
) *(
    0.082*df_diario['dr']
)*(
    (
        (
            df_diario.ang_horario_puesta_sol * np.sin(df_diario.latitud_rad)
        )*(
            np.sin(df_diario.declinacion)
        )
    )+(
        np.cos(df_diario.latitud_rad)*np.cos(df_diario.declinacion)*np.sin(df_diario.ang_horario_puesta_sol)
    )
)


df_diario['radiacion_solar_piranometro'] = 0.0864*df_diario.radiacion_diaria_promedio

df_diario['constante_psicometrica'] =(
    (
        pow(
            (
                (293-(0.0065*df_diario.altitud))/293
            )
            , 5.26
        )
    )*101.3
)*0.000665

df_diario['pendiente_curva_presion_satu_vapor'] =(
    4089*(
        (
            0.6108*(np.exp((17.27*df_diario.temperatura_promedio_diaria)/(df_diario.temperatura_promedio_diaria+237.3))))/pow((df_diario.temperatura_promedio_diaria+273.3), 2)))

df_diario['presion_saturacion_vapor'] =((0.6108*np.exp((17.27*df_diario.temperatura_min_diaria)/(df_diario.temperatura_min_diaria +237.3)))+(0.6108*np.exp((17.27*df_diario.temperatura_max_diaria)/(df_diario.temperatura_max_diaria+237.3))))/2

df_diario['presion_real_vapor'] = (
    (
        0.6108*np.exp(
            (
                17.27*df_diario.temperatura_max_diaria
            )/(
                df_diario.temperatura_max_diaria +237.3
            )
        )
    ) * (
        df_diario.humedad_relativa_min_diaria /100
    ) + (
        0.6108*np.exp(
            (
                17.27*df_diario.temperatura_min_diaria
            )/ (
                df_diario.temperatura_min_diaria+273.3
            )
        )
    ) * (
        df_diario.humedad_relativa_max_diaria/100
    )
)/2

df_diario['deficit_presion_vapor'] = df_diario['presion_saturacion_vapor'] - df_diario['presion_real_vapor']

df_diario['rns']=df_diario.radiacion_solar_piranometro * 0.77

df_diario['rnl']= (
    (
        (
            (
                0.000000004903*(
                    pow(
                        (df_diario.temperatura_min_diaria+273.16),
                        4
                    )
                )
            ) + (
                (
                    0.000000004903 * (
                        pow(
                            (df_diario.temperatura_max_diaria+273.16),
                            4
                        )
                    )
                )
            )
        ) / 2
    ) * (
        0.34 -(
            0.14 *np.sqrt(df_diario.presion_real_vapor)
        )
    ) * (
        1.35 * (
            df_diario.radiacion_solar_piranometro /(
                (
                    0.75+2*(df_diario.altitud/100000)
                )* df_diario['ra_MJm-2day-1']
            )
        ) -0.35
    )
)
            

df_diario['rn']=df_diario['rns']-df_diario['rnl']

df_diario['velocidad_viento_altura_standar'] = (
    (4.87)/(
        (np.log((67.8*10)-5.42))
    )
)*(
    (df_diario.velocidad_viento_media_diario*1000)/3600
)

df_diario['FAO-Penman-Montieth'] = (
    (
        0.408*df_diario.pendiente_curva_presion_satu_vapor*df_diario.rn
    )+(
        df_diario.constante_psicometrica*(
            900/(df_diario.temperatura_promedio_diaria + 273)
        )*df_diario.velocidad_viento_altura_standar * df_diario.deficit_presion_vapor
    )
) / (
    df_diario.pendiente_curva_presion_satu_vapor + (
        df_diario.constante_psicometrica + (
            1+(0.34 + df_diario.velocidad_viento_altura_standar)
        )
    )
)
    

In [ ]:
df_diario['deficit']= df_diario.lluvia_diaria-df_diario['FAO-Penman-Montieth']